In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

In [ ]:
%%manim -qm -v WARNING SPR

from manim import *
import networkx as nx

from manim import *
import networkx as nx
import random

class SPR(Scene):
    def construct(self):
        # Step 1: Define Instances (Squares) and Bumps (Circles)
        instances = {
            "A": Square(0.5, color=BLUE).move_to([-4, 3, 0]),
            "B": Square(0.5, color=GREEN).move_to([-2, 3, 0]),
            "C": Square(0.5, color=RED).move_to([0, 3, 0]),
            "D": Square(0.5, color=ORANGE).move_to([2, 3, 0]),
            "E": Square(0.5, color=PINK).move_to([4, 3, 0])
        }

        bumps = {
            "X": Circle(radius=0.3, color=WHITE, fill_opacity=1).move_to([-3, -3, 0]),
            "Y": Circle(radius=0.3, color=WHITE, fill_opacity=1).move_to([3, -3, 0]),
        }

        # Step 2: Define Connection Nodes (Small Circles)
        nodes = {
            "N1": Dot(point=[-4, 1, 0], color=YELLOW),
            "N2": Dot(point=[-2, 1, 0], color=YELLOW),
            "N3": Dot(point=[0, 1, 0], color=YELLOW),
            "N4": Dot(point=[2, 1, 0], color=YELLOW),
            "N5": Dot(point=[4, 1, 0], color=YELLOW),
            "M1": Dot(point=[-3, -1, 0], color=YELLOW),
            "M2": Dot(point=[3, -1, 0], color=YELLOW),
        }

        # Merge all elements into a single lookup dictionary
        elements = {**instances, **nodes, **bumps}

        # Step 3: Create Graph with Resistances (Weights)
        G = nx.Graph()
        edges = [
            ("A", "N1", 2), ("N1", "N2", 1), ("N2", "B", 2),
            ("B", "N3", 3), ("N3", "C", 1), ("C", "N4", 2),
            ("N4", "D", 3), ("D", "N5", 2), ("N5", "E", 1),
            ("N1", "M1", 3), ("N3", "M2", 3),
            ("M1", "X", 2), ("M2", "Y", 2),
        ]

        for u, v, w in edges:
            G.add_edge(u, v, weight=w)

        # Step 4: Draw All Elements
        self.play(*[Create(inst) for inst in instances.values()])
        self.play(*[Create(bump) for bump in bumps.values()])
        self.play(*[Create(node) for node in nodes.values()])

        # Label Instances
        instance_labels = [Text(k, font_size=24).next_to(v, UP) for k, v in instances.items()]
        self.play(*[Write(label) for label in instance_labels])

        # Draw Nets (Edges) with Resistance Labels
        edges_visual = []
        edge_labels = []
        for u, v, w in edges:
            if u in elements and v in elements:
                edge = Line(elements[u].get_center(), elements[v].get_center(), color=GRAY)
                edges_visual.append(edge)
                label = Text(str(w), font_size=20, color=WHITE).move_to((elements[u].get_center() + elements[v].get_center()) / 2)
                edge_labels.append(label)

        self.play(*[Create(edge) for edge in edges_visual])
        self.play(*[Write(label) for label in edge_labels])

        self.wait(1)

        # Step 5: Compute Shortest Path using Dijkstra
        shortest_path = nx.shortest_path(G, source="A", target="X", weight="weight")

        # Step 6: Highlight All Choices Before Selecting Final Path
        for i in range(len(shortest_path) - 1):
            start = elements[shortest_path[i]].get_center()
            end = elements[shortest_path[i+1]].get_center()
            all_choices = [Line(start, elements[v].get_center(), color=random.choice([BLUE, GREEN, RED, ORANGE, PINK]))
                           for v in G.neighbors(shortest_path[i])]
            self.play(*[Create(choice) for choice in all_choices], run_time=0.5)
            self.wait(0.3)
            self.play(*[FadeOut(choice) for choice in all_choices], run_time=0.3)
            chosen_edge = Line(start, end, color=YELLOW)
            self.play(Create(chosen_edge), run_time=0.8)

        self.wait(2)




In [ ]:
%%manim -qm -v WARNING SPR

from manim import *
import networkx as nx
import random
import numpy as np

class SPR(Scene):
    def construct(self):
        # Step 1: Define Instances (Squares) and Bumps (Circles)
        instances = {
            "A": Square(0.5, color=BLUE).move_to([-4, 3, 0]),
            "B": Square(0.5, color=GREEN).move_to([-2, 3, 0]),
            "C": Square(0.5, color=RED).move_to([0, 3, 0]),
            "D": Square(0.5, color=ORANGE).move_to([2, 3, 0]),
            "E": Square(0.5, color=PINK).move_to([4, 3, 0])
        }

        bumps = {
            "X": Circle(radius=0.3, color=WHITE, fill_opacity=1).move_to([-3, -4, 0]),
            "Y": Circle(radius=0.3, color=WHITE, fill_opacity=1).move_to([3, -4, 0]),
        }

        virtual_ground = "VG"

        # Step 2: Define Nodes in a Grid Layout
        grid_size = 5
        node_spacing = 2
        nodes = {}
        for i in range(grid_size):
            for j in range(grid_size):
                node_name = f"N{i}{j}"
                nodes[node_name] = Dot(point=[-4 + j * node_spacing, 2 - i * node_spacing, 0], color=YELLOW, radius=0.15)

        # Merge all elements into a single lookup dictionary
        elements = {**instances, **nodes, **bumps}

        # Step 3: Create Graph with Resistances (Weights)
        G = nx.Graph()
        edges = []

        # Connect instances to nearest nodes
        for inst in instances:
            closest_node = min(nodes.keys(), key=lambda n: np.linalg.norm(np.array(instances[inst].get_center()) - np.array(nodes[n].get_center())))
            resistance = random.randint(2, 6)
            edges.append((inst, closest_node, resistance))
            G.add_edge(inst, closest_node, weight=resistance)

        # Connect nodes in a grid pattern
        for i in range(grid_size):
            for j in range(grid_size):
                node = f"N{i}{j}"
                if j < grid_size - 1:  # Connect right
                    right_node = f"N{i}{j+1}"
                    resistance = random.randint(2, 6)
                    edges.append((node, right_node, resistance))
                    G.add_edge(node, right_node, weight=resistance)
                if i < grid_size - 1:  # Connect down
                    down_node = f"N{i+1}{j}"
                    resistance = random.randint(2, 6)
                    edges.append((node, down_node, resistance))
                    G.add_edge(node, down_node, weight=resistance)

        # Connect nodes to bumps with proper resistance and isolation
        for bump in bumps:
            closest_node = min(nodes.keys(), key=lambda n: np.linalg.norm(np.array(bumps[bump].get_center()) - np.array(nodes[n].get_center())))
            resistance = random.randint(3, 7)
            edges.append((closest_node, bump, resistance))
            G.add_edge(closest_node, bump, weight=resistance)

            # Connect bump to virtual ground with low resistance
            edges.append((bump, virtual_ground, 0.00001))
            G.add_edge(bump, virtual_ground, weight=0.00001)

        # Step 4: Draw All Elements
        self.play(*[Create(inst) for inst in instances.values()])
        self.play(*[Create(bump) for bump in bumps.values()])
        self.play(*[Create(node) for node in nodes.values()])

        # Label Instances
        instance_labels = [Text(k, font_size=24).next_to(v, UP * 1.5) for k, v in instances.items()]
        self.play(*[Write(label) for label in instance_labels])

        # Draw Nets (Edges) with Resistance Labels
        edges_visual = []
        edge_labels = []
        for u, v, w in edges:
            if u in elements and v in elements:
                edge = Line(elements[u].get_center(), elements[v].get_center(), color=GRAY)
                edges_visual.append(edge)
                label = Text(str(w), font_size=20, color=WHITE).move_to((elements[u].get_center() + elements[v].get_center()) / 2)
                edge_labels.append(label)

        self.play(*[Create(edge) for edge in edges_visual])
        self.play(*[Write(label) for label in edge_labels])

        self.wait(1)

        # Step 5: Compute and Display Shortest Path for Each Instance
        for inst in instances:
            info_text = Text(f"Starting visualization of SPR for instance {inst}", font_size=24).to_edge(UP * 2)
            self.play(Write(info_text))
            shortest_path = nx.shortest_path(G, source=inst, target=virtual_ground, weight="weight")

            # Highlight all possible paths
            for i in range(len(shortest_path) - 1):
                start = elements[shortest_path[i]].get_center()
                for neighbor in G.neighbors(shortest_path[i]):
                    if neighbor in elements:
                        possible_edge = Line(start, elements[neighbor].get_center(), color=BLUE, stroke_width=2)
                        self.play(Create(possible_edge), run_time=0.3)

            self.wait(0.5)

            # Highlight chosen shortest path
            for i in range(len(shortest_path) - 2):
                start = elements[shortest_path[i]].get_center()
                end = elements[shortest_path[i+1]].get_center()
                chosen_edge = Line(start, end, color=YELLOW, stroke_width=4)
                self.play(Create(chosen_edge), run_time=1.2)

            self.wait(1)
            self.play(FadeOut(info_text))

        self.wait(2)
